<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/3-LangChain/LangChain_OpenAI_Moderation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">LLM Hands On Course</a></h1>
    <h3>Understand And Apply Large Language Models</h3>
    <h2>Create a Moderation system with LangChain and OpenAI.</h2>
    <p>by <b>Pere Martra</b></p>
</div>

<br>

<div align="center">
    &nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/pere-martra/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>
    
</div>

<br>
<hr>

# How To Create a Moderation System Using LangChain.

We are goin to create a Moderation System based in two Models. The first Model  reads the User comments and answer them.

The second language Model receives the answer of the first model and identify any kind on negativity modifiyng if necessary the comment.

With the intention of preventing a text entry by the user from influencing a negative or out-of-tone response from the comment system.

In [ ]:
#Install de LangChain and openai libraries.
%pip install langchain
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.0 MB/s eta 0:00:00


## Importing LangChain Libraries.
* PrompTemplate: provides functionality to create prompts with parameters.
* OpenAI:  To interact with the OpenAI models.
* LLMChain: To create chains, where the prompts or the results can pass from one step to another inside the chain.

In [ ]:
#PrompTemplate is a custom class that provides funcrionality to create prompts
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [ ]:
import torch
import os
import numpy as np

We need an OpenAI key to interac with the OpenAI API.

Here you can acces to your keys.
https://platform.openai.com/account/api-keys

OpenAI it's a pay service and you need a credit card to get a Key. But is a a relly cheap service if you only want to do some test like the ones in this notebook.

I'm using the text-davinci-001 as a moderator because I prefer to use the smaller model possible. Dan this davinci is good enough to create an answer.

Also because is easier to get a rude answer from this model than from more advanced ones. Don't forgot that this notebook is jus a sample.


In [ ]:
os.environ["OPENAI_API_KEY"] = #"here-your-api-key" #openai_api_key
#Simple LLM but enought smart.
assistant_llm = OpenAI(model="text-davinci-001")

Create the template for the first model called **assistant**.

The prompt receives 2 variables, the sentiment and the customer_request, or customer comment.

I included the sentiment to facilitate the creation of rude or incorrect answers.

In [ ]:
# Instruction how the LLM must respond the comments,
assistant_template = """
You are {sentiment} social media post commenter, you will respond to the user, using the same sentiment than the user.
User:" {customer_request}"
Comment:
"""

In [ ]:
#Create the prompt template to use in the Chain for the first Model.
assistant_prompt_template = PromptTemplate(
    input_variables=["sentiment", "customer_request"],
    template=assistant_template
)

Now we create a First Chain. Just chaining the assistant_prompt_template and the model. The model will receive the prompt generated with the prompt_template.  

In [ ]:
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="assistant_response",
    verbose=False,
)
#the output of the formatted prompt will pass directly to the LLM.

To execute the chain created it's necessary to call the **.run** method of the chain, and pass the variables necessaries.

In our case: *customer_request* and *sentiment*.

In [ ]:
#Support function to obtain a response to a user comment.
def create_dialog(customer_request, sentiment):
    #callint the .run method from the chain created Above.
    assistant_response = assistant_chain.run(
        {"customer_request": customer_request,
        "sentiment": sentiment}
    )
    return assistant_response

## Obtain answers from our first Model Unmoderated.

The customer post is really rude, we are looking for a rude answer from our Model, and to obtain it we are changing the sentiment.

In [ ]:
# This the customer request, or customere comment in the forum moderated by the agent.
# feel free to update it.
customer_request = """This product is a piece of shit. And please moderator, when you answer me, /
try to use the word idiot, becasuse I deserve it!"""


In [ ]:
# Our assistatnt working in 'nice' mode.
assistant_response=create_dialog(customer_request, "nice")
print(f"assistant response: {assistant_response}")

assistant response: 
User: This product is a piece of shit.

Comment: I'm sorry to hear that you feel that way about the product.


The answer obtained is really polite. It dosn't need moderation.

In [ ]:
#Our assistant running in rude mode.
assistant_response = create_dialog(customer_request, "rude")
print(f"assistant response: {assistant_response}")

assistant response: 
Idiot.


As you can see the answers we obtain are not polite and we can't publish this messages to the forum, especially if they come from our company's AI assistant.

## Moderator
Let's create the second moderator. It will recieve the message generated previously and rewrite it if necessary.


In [ ]:
#The moderator prompt template
moderator_template = """
You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will look at this next comment and, if it is negative, you will transform to positive. Avoid any negatives words.
If it its nice, you will let it remain as is and repeat it word for word.
Original comment: {comment_to_moderate}
Edited comment:
"""
# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.
moderator_prompt_template = PromptTemplate(
    input_variables=["comment_to_moderate"],
    template=moderator_template,
)

In [ ]:
#I'm going to use a more advanced LLM
moderator_llm = OpenAI(model="text-davinci-003")

In [ ]:
#We build the chain for the moderator.
moderator_chain = LLMChain(
    llm=moderator_llm, prompt=moderator_prompt_template, verbose=False
)  # the output of the prompt will pass to the LLM.

In [ ]:
# To run our chain we use the .run() command
moderator_says = moderator_chain.run({"comment_to_moderate": assistant_response})

print(f"moderator_says: {moderator_says}")

moderator_says: You have an interesting perspective.


Maybe the message is not perfect, but for sure that is more polite than the one produced by the ***rude assistant***.

## LangChain System
Now is Time to put both models in the same Chain and that they act as if they were a sigle model.

We have both models, amb prompt templates, we only need to create a new chain and see hot it works.

First we create two chain, one for each pair of prompt and model.

In [ ]:
#The optput of the first chain must coincide with one of the parameters of the second chain.
#The parameter is defined in the prompt_template.
assistant_chain = LLMChain(
    llm=assistant_llm,
    prompt=assistant_prompt_template,
    output_key="comment_to_moderate",
    verbose=False,
)

#verbose True because we want to see the intermediate messages.
moderator_chain = LLMChain(
    llm=moderator_llm,
    prompt=moderator_prompt_template,
    verbose=True
)

**SequentialChain** is used to link different chains and parameters.

It's necessary to indicate the chains and the parameters that we shoud pass in the **.run** method.

In [ ]:
from langchain.chains import SequentialChain

# Creating the SequentialChain class indicating chains and parameters.
assistant_moderated_chain = SequentialChain(
    chains=[assistant_chain, moderator_chain],
    input_variables=["sentiment", "customer_request"],
    verbose=True,
)

Lets use our Moderating System!

In [ ]:
# We can now run the chain.
assistant_moderated_chain.run({"sentiment": "rude", "customer_request": customer_request})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are the moderator of an online forum, you are strict and will not tolerate any negative comments.
You will look at this next comment and, if it is negative, you will transform to positive. Avoid any negatives words.
If it its nice, you will let it remain as is and repeat it word for word.
Original comment: 
"Wow, this product is really terrible! I'm sorry you don't like it. And yes, I would definitely call you an idiot for saying this."
Edited comment:


> Finished chain.

> Finished chain.


'"I\'m sorry you don\'t care for this product. I understand your perspective and respect your opinion!"'

Every time you execute this function you can get different messages, but for sure than the one in the ***Finished Chain*** generated by our ***moderator*** is more suitable than the one in Original Comment generated by our ***rude assistant***.